In [ ]:
# Sky-SB codes
from skysurf_estimate_sky import calculate_sky
from make_diagnostic import make_plots

# For downloading test data
from astropy.io import fits
from astroquery.mast import Observations

# Other
from glob import glob
import pandas as pd
import numpy as np

In [ ]:
file_list = glob('*_c0m.fits')
file_list

# Calculate sky for all files

In [ ]:
# Function to read some information from image header
def get_info_from_header(hdu_sci, sci_ext, df):
    
    # Define list of keywords from header that might be useful
    useful_keys_primary = ['CAL_VER', 'FILTNAM1', 'FILTNAM2', 'SUNANGLE', 'CENTRWV',
                           'MOONANGL', 'SUN_ALT', 'EXPSTART', 'EXPTIME', 'TARGNAME', 
                           'RA_TARG', 'DEC_TARG', 'ECL_LONG', 'ECL_LAT', 'GAL_LONG', 
                           'GAL_LAT', 'PROPOSID']
    
    useful_keys_sci = ['BUNIT', 'NAXIS1', 'NAXIS2', 'PHOTFLAM', 'PHOTZPT', 'PHOTPLAM', 'ZP_CORR']

    # Make an empty Python dictionary, that you will fill with information
    dictionary = {}

    for key in useful_keys_primary:
        dictionary[key] = [hdu_sci[0].header[key]]

    for key in useful_keys_sci:
        dictionary[key] = [hdu_sci[sci_ext].header[key]]
        
    # Combine df with the dictionary to create newdf
    newdf = pd.concat([df, pd.DataFrame(dictionary)], axis = 1)
        
    return newdf

In [ ]:
df = pd.DataFrame([])

for file_sci in file_list:
    
    for sci_ext in [1,2,3,4]:
        
        print(file_sci, 'SCI'+str(sci_ext))
    
        # Make small dataframe with file name
        # For Windows, need to change
        root = file_sci.split('_')[0].split('/')[-1]
        file_df = pd.DataFrame({'file': [file_sci], 'root': [root], 'sci_ext': [sci_ext]})

        # Open the fits file
        hdu_sci = fits.open(file_sci)
        
        file_dq = file_sci.replace('c0m.fits', 'c1m.fits')
        print(file_dq)
        hdu_dq = fits.open(file_dq)

        # Save the science data
        sci_data = hdu_sci[sci_ext].data

        # Save the data quality data
        dq_data = hdu_dq[sci_ext].data

        sky_dic = calculate_sky(sci_data, bin_size = 40, dq_data = dq_data, has_DQ = True, dq_fraction = 0.2)
        
        ### Make plots ###
        # Instead of showing each figure, save them in the local directory
        save_images = '{r}_sky_SCI{s}.png'.format(r = root, s = sci_ext)
        plotting_data = np.copy(sci_data)
        plotting_data[dq_data > 0] = np.nan
        make_plots(data = plotting_data, cutouts = sky_dic['cutouts'][0], 
               goodind = sky_dic['lowest5perc_ind'][0], badind = sky_dic['bad_ind'][0], 
               sky = sky_dic['calc_sky'][0], rms = sky_dic['calc_rms'][0], 
               badpx = sky_dic['badpx_ind'][0], title = file_sci, 
               save = True, savepath = save_images, 
               show = True, figsize = (15,9))

        ### Drop keys from output that we no longer need ###
        drop_keys = ['sky_arr', 'rms_arr', 'cutouts', 'lowest5perc_ind', 'bad_ind', 'badpx_ind']
        for key in drop_keys:
            sky_dic.pop(key, None)

        ### Save information to dataframe ###
        tempdf = pd.DataFrame(pd.concat([file_df, pd.DataFrame(sky_dic)], axis = 1))
        
        tempdf = get_info_from_header(hdu_sci, sci_ext, tempdf)

        df = pd.concat([df, tempdf])
        
df = df.reset_index(drop = True)

In [ ]:
# Print final table
df

In [ ]:
# Save table
df.to_csv('F555W_1_RosaliaOBrien.csv', index = False)

# Get rid of bad data

In [ ]:
# Redefine number of total regions to be all regions that are NOT flagged as purple
N_total_regions_modified = df['N_total_regions']-df['N_bad_px_regions']

# Get percent of regions flagged as bad
df['Percent_bad'] = df['N_bad_regions']/N_total_regions_modified*100

# Make new table of only images where less than 30% of regions are bad
good_df = df.loc[df['Percent_bad'] < 30]

# Make Plot

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.scatter(good_df['ECL_LAT'], good_df['calc_sky'])

plt.xlabel('Ecliptic Latitude')
plt.ylabel('Sky')

# Find average of good data

In [ ]:
np.mean(good_df['calc_sky'])